#### Creating an LLM Agent

Our LLM Agent will call 3 tools - execute_shell_comand, read_file, and file_write.

In [72]:
!pip install --force-reinstall -v -q "openai==1.55.3"

  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6

In [73]:
pip install python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [74]:
import os

with open("key.env", "r") as file:
    openai_api_key = file.read().strip()

os.environ["OPENAI_API_KEY"] = openai_api_key

with open("key.env", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

In [75]:
import openai
import json
import subprocess
from typing import List, Dict, Any
import shlex
from openai import OpenAI

In [76]:
class LLMAgent:
    #initialize the assistance
    def __init__(self, client, model):
        self.model = model
        self.client = client

        #define the 3 tools
        self.tools = [
            {
                "type": "function",
                "function": {
                    #shell_command tool
                    "name": "execute_shell_command",
                    "description": "Execute a shell command and return its output",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {
                                "type": "string",
                                "description": "The shell command to execute"
                            }
                        },
                        "required": ["command"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    #read_file tool
                    "name": "read_file",
                    "description": "Read contents of a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {
                                "type": "string",
                                "description": "Path to the file to read"
                            }
                        },
                        "required": ["file_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    #file_write tool
                    "name": "write_file",
                    "description": "Write content to a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {
                                "type": "string",
                                "description": "Path to the file to write"
                            },
                            "content": {
                                "type": "string",
                                "description": "Content to write to file"
                            },
                            "mode": {
                                "type": "string",
                                "description": "Write mode: 'w' for overwrite, 'a' for append",
                                "enum": ["w", "a"],
                                "default": "w"
                            },
                        },
                        "required": ["file_path", "content"]
                    }
                }
            }
        ]

    #write_file function
    def write_file(self, file_path: str, content: str, mode: str = 'w'):
        """Write content to a file."""
        #write content to file
        try:
            #w=overwrite and a=append
            if mode not in ['w', 'a']:
                raise ValueError("The mode must be w or a.")

            abs_path = os.path.abspath(file_path)

            with open(abs_path, mode) as f:
                f.write(content)

            return {
                "success": True,
                "message": f"Successfully wrote to {file_path}",
                "bytes_written": len(content)
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }


    #execute_shell_command function
    def execute_shell_command(self, command:str) -> Dict[str, Any]:
        """Execute a shell command and return its output."""
        try:
            #shlex parses the command
            args = shlex.split(command)

            #execute command and get output
            result = subprocess.run(
                args,
                capture_output = True,
                text=True,
                shell=False
            )
            return {
                "success": True,
                "stdout": result.stdout,
                "stderr": result.stderr,
                "return_code": result.returncode
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    #read_file function
    def read_file(self, file_path: str) -> Dict[str, Any]:
        """Read a file and return its contents."""
        try:
            with open(file_path, 'r') as f:
                content = f.read()
            return{
                "success": True,
                "content": content
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    #call tools
    def process_tool_call(self, tool_call:Dict) -> Dict[str, Any]:
        """Process a tool call from the API response."""
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "execute_shell_command":
            return self.execute_shell_command(function_args["command"])
        elif function_name == "read_file":
            return self.read_file(function_args["file_path"])
        elif function_name == "write_file":
            mode = function_args.get("mode", "w")
            return self.write_file(
                function_args["file_path"],
                function_args["content"],
                mode
            )
        else:
            return {"success": False, "error": f"Unknown function: {function_name}"}

    def chat(self, user_message: str, verbose=False) -> str:
        """Main chat function that processes user input and returns assistant response."""
        completions = []
        messages = [{"role": "user", "content": user_message}]

        try:
            # Get initial response from OpenAI
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )

            # completions.append(completion)
            message = completion.choices[0].message

            # Process tool calls if any
            while message.tool_calls:
                messages.append(message)

                # Process each tool call
                for tool_call in message.tool_calls:
                    result = self.process_tool_call(tool_call)

                    # Add tool result to messages
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result)
                    })

                # Get next response from OpenAI
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    tools=self.tools,
                    tool_choice="auto"
                )
                # completions.append(completion)
                message = completion.choices[0].message

            if verbose:
                return message.content, messages#, completions
            else:
                return message.content

        except Exception as e:
            return f"Error: {str(e)}"

#### Now, lets create the agent

In [87]:
from openai import OpenAI

assistant = LLMAgent(
    client = OpenAI(),
    model = "gpt-4o-mini"
    )

In [89]:
result = assistant.chat('Get the names of the files in the current directory', verbose=True)
result

'Error: Connection error.'